In [4]:
import pandas as pd
import numpy as np 

# reading train data
train_data = pd.read_csv('pp5i_train.gr.csv')
print("orig: ", train_data.shape)


# droping the index column
# train_data.drop(['SNO'], axis=1, inplace=True)
train_data.set_index(keys='SNO', inplace=True)
print("after set index: ", train_data.shape)



#  Threshold data
train_data[train_data>16000] = 16000
train_data[train_data<20] = 20



# calculating and removing fold differences
train_data['fold_differences'] = train_data.max(axis=1) / train_data.min(axis=1)
train_data.drop(train_data[ train_data['fold_differences']<2 ].index, axis=0, inplace=True)
train_data.drop(['fold_differences'], axis=1, inplace=True)
print("after fold differences: ", train_data.shape)
train_data.loc[len(train_data)] = "MED"

# reading class labels and adding them to the train data
labels = pd.read_table('pp5i_train_class.txt')
train_data.iloc[-1, :] = labels.T.values
# for idx, item in enumerate(train_data.iloc[-1, :]):
#     train_data.iloc[-1, idx] = item[0]

train_data = train_data.T



print(train_data)

orig:  (7070, 70)
after set index:  (7070, 69)
after fold differences:  (6413, 69)
SNO A28102_at AB000114_at AB000115_at AB000220_at AB000409_at AB000449_at  \
1          30          22          29          76         167          32   
2          46          31          70         208         211          20   
3          31          20          20         244         179         229   
4          31          20          20          39         119          20   
5          26          26          20          85         161          34   
..        ...         ...         ...         ...         ...         ...   
65         68          21          35          30         131          20   
66         77          22          30          31         132          20   
67         56          25          65          31         158          20   
68         41          21          32          27         164          20   
69         38          20          25          20         172         

### MED

In [5]:
med = train_data[ train_data.iloc[:, -1] == "MED" ]
med = med.T

med.drop(med.index[-1], inplace=True)
# med = med.T

med_inverse = train_data[ train_data.iloc[:, -1] != "MED"]
med_inverse = med_inverse.T
med_inverse.drop(med_inverse.index[-1], inplace=True)
# med_inverse = med_inverse.T


# print(med.shape, med_inverse.shape)

med["mean"] = med.mean(axis=1, skipna=True)
med["std"] = med.std(axis=1, skipna=True)
# print(med.head(3))

med_inverse["mean"] = med_inverse.mean(axis=1, skipna=True)
med_inverse["std"] = med_inverse.std(axis=1, skipna=True)
# print(med_inverse.head(3))

nume = med.loc[:,'mean'] - med_inverse.loc[:,'mean']
deno = np.sqrt( med.loc[:,'std'].pow(2)/40  + med_inverse.loc[:,'std'].pow(2)/30)

med.loc[:, 'T'] = nume / deno

med.drop(labels=['mean', 'std'], axis=1, inplace=True)


med.sort_values(['T'], ascending=False, na_position='last', inplace=True)
# med['MED'] = "MED"
med.drop(labels='T', axis=1, inplace=True)

top_meds = {}
top_meds_index = {}
N = [2, 4, 6, 8, 10, 12, 15, 20, 25, 30]
for item in N:
    top_meds[item] = med.iloc[:item, :]
    top_meds[item].sort_index(axis=0, inplace=True, na_position='last', ascending=True)
    top_meds_index[item] = list(top_meds[item].index)
    
# top_med = med.iloc[:30, :]
# top_med.sort_index(axis=0, inplace=True, na_position='last', ascending=True)
# top_med.to_csv('med.csv')
# med_index = list(top_med.index)
print(top_meds_index)

{2: ['M31303_rna1_at', 'U16954_at'], 4: ['D80004_at', 'M31303_rna1_at', 'U16954_at', 'U79262_at'], 6: ['D50310_at', 'D80004_at', 'M31303_rna1_at', 'M93119_at', 'U16954_at', 'U79262_at'], 8: ['D50310_at', 'D80004_at', 'M31303_rna1_at', 'M93119_at', 'U09087_s_at', 'U16954_at', 'U79262_at', 'Z69915_at'], 10: ['D50310_at', 'D80004_at', 'M31303_rna1_at', 'M93119_at', 'U09087_s_at', 'U16954_at', 'U30521_at', 'U79262_at', 'X73358_s_at', 'Z69915_at'], 12: ['D50310_at', 'D80004_at', 'L37792_at', 'M31303_rna1_at', 'M93119_at', 'U09087_s_at', 'U16954_at', 'U30521_at', 'U79248_at', 'U79262_at', 'X73358_s_at', 'Z69915_at'], 15: ['D50310_at', 'D80004_at', 'L37792_at', 'M31303_rna1_at', 'M80629_at', 'M93119_at', 'S82024_at', 'U09087_s_at', 'U16954_at', 'U19878_at', 'U30521_at', 'U79248_at', 'U79262_at', 'X73358_s_at', 'Z69915_at'], 20: ['D50310_at', 'D78012_at', 'D80004_at', 'L11931_at', 'L37792_at', 'M31303_rna1_at', 'M80629_at', 'M93119_at', 'S82024_at', 'U09087_s_at', 'U16954_at', 'U18271_cds3_s_a

C:\Users\Cypher\anaconda3\lib\site-packages\pandas\core\frame.py:6393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().sort_index(


### MGL

In [6]:
mgl = train_data[ train_data.iloc[:, -1] == "MGL" ]
mgl = mgl.T
mgl.drop(mgl.index[-1], inplace=True)
# mgl = mgl.T

mgl_inverse = train_data[ train_data.iloc[:, -1] != "MGL"]
mgl_inverse = mgl_inverse.T
mgl_inverse.drop(mgl_inverse.index[-1], inplace=True)
# mgl_inverse = mgl_inverse.T

# print(mgl.shape, mgl_inverse.shape)

mgl["mean"] = mgl.mean(axis=1, skipna=True)
mgl["std"] = mgl.std(axis=1, skipna=True)
# print(mgl.head(3))

mgl_inverse["mean"] = mgl_inverse.mean(axis=1, skipna=True)
mgl_inverse["std"] = mgl_inverse.std(axis=1, skipna=True)
# print(mgl_inverse.head(3))


nume = mgl.loc[:,'mean'] - mgl_inverse.loc[:,'mean']
deno = np.sqrt( mgl.loc[:,'std'].pow(2)/7  + mgl_inverse.loc[:,'std'].pow(2)/63)

mgl.loc[:, 'T'] = nume / deno

mgl.drop(labels=['mean', 'std'], axis=1, inplace=True)
mgl.sort_values(['T'], ascending=False, na_position='last', inplace=True)
# mgl['MGL'] = "MGL"
mgl.drop(labels='T', axis=1, inplace=True)


# top_mgl = mgl.iloc[:30, :]
# top_mgl.sort_index(axis=0, inplace=True, na_position='last', ascending=True)
# # top_mgl.to_csv('mgl.csv')
# mgl_index = list(top_mgl.index)
# mgl_index
top_mgls = {}
top_mgls_index = {}
N = [2, 4, 6, 8, 10, 12, 15, 20, 25, 30]
for item in N:
    top_mgls[item] = mgl.iloc[:item, :]
    top_mgls[item].sort_index(axis=0, inplace=True, na_position='last', ascending=True)
    top_mgls_index[item] = list(top_mgls[item].index)
print(top_mgls_index)

{2: ['V01515_cds1_at', 'X00371_rna1_at'], 4: ['D85815_at', 'M20137_at', 'V01515_cds1_at', 'X00371_rna1_at'], 6: ['D85815_at', 'M20137_at', 'U17760_rna1_at', 'V01515_cds1_at', 'X00371_rna1_at', 'X52011_at'], 8: ['D85815_at', 'L25270_at', 'M20137_at', 'M83308_at', 'U17760_rna1_at', 'V01515_cds1_at', 'X00371_rna1_at', 'X52011_at'], 10: ['D85815_at', 'L25270_at', 'L36463_at', 'M20137_at', 'M83308_at', 'U17760_rna1_at', 'V01515_cds1_at', 'X00371_rna1_at', 'X03066_at', 'X52011_at'], 12: ['D85815_at', 'L25270_at', 'L36463_at', 'L43338_at', 'M20137_at', 'M83308_at', 'U17760_rna1_at', 'U70321_at', 'V01515_cds1_at', 'X00371_rna1_at', 'X03066_at', 'X52011_at'], 15: ['D85815_at', 'HG4490-HT4876_f_at', 'L25270_at', 'L36463_at', 'L41351_at', 'L43338_at', 'M20137_at', 'M83308_at', 'U17760_rna1_at', 'U70321_at', 'U78551_at', 'V01515_cds1_at', 'X00371_rna1_at', 'X03066_at', 'X52011_at'], 20: ['D85815_at', 'HG3286-HT3463_at', 'HG4490-HT4876_f_at', 'HG881-HT881_at', 'L25270_at', 'L36463_at', 'L41351_at',

### RHB

In [7]:

rhb = train_data[ train_data.iloc[:, -1] == "RHB" ]
rhb = rhb.T
rhb.drop(rhb.index[-1], inplace=True)
# rhb = rhb.T

rhb_inverse = train_data[ train_data.iloc[:, -1] != "RHB"]
rhb_inverse = rhb_inverse.T
rhb_inverse.drop(rhb_inverse.index[-1], inplace=True)
# rhb_inverse = rhb_inverse.T

# print(rhb.shape, rhb_inverse.shape)

rhb["mean"] = rhb.mean(axis=1, skipna=True)
rhb["std"] = rhb.std(axis=1, skipna=True)
# print(rhb.head(3))

rhb_inverse["mean"] = rhb_inverse.mean(axis=1, skipna=True)
rhb_inverse["std"] = rhb_inverse.std(axis=1, skipna=True)
# print(rhb_inverse.head(3))

nume = rhb.loc[:,'mean'] - rhb_inverse.loc[:,'mean']
deno = np.sqrt( rhb.loc[:,'std'].pow(2)/7  + rhb_inverse.loc[:,'std'].pow(2)/63)

rhb.loc[:, 'T'] = nume / deno
rhb.drop(labels=['mean', 'std'], axis=1, inplace=True)

rhb.sort_values(['T'], ascending=False, na_position='last', inplace=True)
# rhb['RHB'] = "RHB"
rhb.drop(labels='T', axis=1, inplace=True)

# top_rhb = rhb.iloc[:30, :]
# top_rhb.sort_index(axis=0, inplace=True, na_position='last', ascending=True)
# # top_rhb.to_csv('rhb.csv')
# rhb_index = list(top_rhb.index)
# rhb_index
top_rhbs = {}
top_rhbs_index = {}
N = [2, 4, 6, 8, 10, 12, 15, 20, 25, 30]
for item in N:
    top_rhbs[item] = rhb.iloc[:item, :]
    top_rhbs[item].sort_index(axis=0, inplace=True, na_position='last', ascending=True)
    top_rhbs_index[item] = list(top_rhbs[item].index)
print(top_rhbs_index)

{2: ['D42041_at', 'J03191_at'], 4: ['D42041_at', 'HG3514-HT3708_at', 'J03191_at', 'M57464_s_at'], 6: ['D42041_at', 'D45248_at', 'HG3514-HT3708_at', 'J03191_at', 'M57464_s_at', 'X54304_at'], 8: ['D42041_at', 'D45248_at', 'HG3514-HT3708_at', 'J03191_at', 'L06132_at', 'M57464_s_at', 'X04106_at', 'X54304_at'], 10: ['D14043_at', 'D42041_at', 'D45248_at', 'HG3514-HT3708_at', 'J03191_at', 'L06132_at', 'M57464_s_at', 'U33818_at', 'X04106_at', 'X54304_at'], 12: ['D14043_at', 'D42041_at', 'D45248_at', 'HG3514-HT3708_at', 'J03191_at', 'L06132_at', 'M57464_s_at', 'U33818_at', 'X04106_at', 'X15187_at', 'X54304_at', 'Z97074_at'], 15: ['D14043_at', 'D31890_at', 'D42041_at', 'D45248_at', 'HG3514-HT3708_at', 'J03191_at', 'L06132_at', 'M57464_s_at', 'U33818_at', 'U47621_at', 'X04106_at', 'X15187_at', 'X54304_at', 'Y07604_at', 'Z97074_at'], 20: ['D14043_at', 'D31890_at', 'D42041_at', 'D45248_at', 'D79997_at', 'D83174_s_at', 'HG3514-HT3708_at', 'J03191_at', 'L06132_at', 'L23333_s_at', 'M34338_s_at', 'M574

In [8]:
epd = train_data[ train_data.iloc[:, -1] == "EPD" ]
epd = epd.T
epd.drop(epd.index[-1], inplace=True)
# epd = epd.T

epd_inverse = train_data[ train_data.iloc[:, -1] != "EPD"]
epd_inverse = epd_inverse.T
epd_inverse.drop(epd_inverse.index[-1], inplace=True)
# epd_inverse = epd_inverse.T

# print(epd.shape, epd_inverse.shape)

epd["mean"] = epd.mean(axis=1, skipna=True)
epd["std"] = epd.std(axis=1, skipna=True)
# print(epd.head(3))

epd_inverse["mean"] = epd_inverse.mean(axis=1, skipna=True)
epd_inverse["std"] = epd_inverse.std(axis=1, skipna=True)
# print(epd_inverse.head(3))

nume = epd.loc[:,'mean'] - epd_inverse.loc[:,'mean']
deno = np.sqrt( epd.loc[:,'std'].pow(2)/10  + epd_inverse.loc[:,'std'].pow(2)/60)

epd.loc[:, 'T'] = nume / deno
epd.drop(labels=['mean', 'std'], axis=1, inplace=True)
epd.sort_values(['T'], ascending=False, na_position='last', inplace=True)
# epd['EPD'] = "EPD"
epd.drop(labels='T', axis=1, inplace=True)
# top_epd = epd.iloc[:30, :]
# top_epd.sort_index(axis=0, inplace=True, na_position='last', ascending=True)
# # top_epd.to_csv('epd.csv')
# epd_index = list(top_epd.index)
# print(top_rhbs_index)
top_epds = {}
top_epds_index = {}
N = [2, 4, 6, 8, 10, 12, 15, 20, 25, 30]
for item in N:
    top_epds[item] = epd.iloc[:item, :]
    top_epds[item].sort_index(axis=0, inplace=True, na_position='last', ascending=True)
    top_epds_index[item] = list(top_epds[item].index)
print(top_epds_index)

{2: ['D14662_at', 'X76105_at'], 4: ['D00763_at', 'D14662_at', 'U07550_at', 'X76105_at'], 6: ['D00763_at', 'D14662_at', 'L36818_at', 'U06155_at', 'U07550_at', 'X76105_at'], 8: ['D00763_at', 'D14662_at', 'L36818_at', 'M75099_at', 'U06155_at', 'U07550_at', 'X65784_s_at', 'X76105_at'], 10: ['D00763_at', 'D14662_at', 'L36818_at', 'M19645_at', 'M75099_at', 'U06155_at', 'U07550_at', 'U91930_at', 'X65784_s_at', 'X76105_at'], 12: ['D00763_at', 'D14662_at', 'L36818_at', 'M19645_at', 'M69066_at', 'M75099_at', 'U06155_at', 'U07550_at', 'U66879_at', 'U91930_at', 'X65784_s_at', 'X76105_at'], 15: ['D00763_at', 'D14662_at', 'L36818_at', 'M19645_at', 'M69066_at', 'M75099_at', 'S79873_s_at', 'U01923_at', 'U06155_at', 'U07550_at', 'U66879_at', 'U91930_at', 'X03363_s_at', 'X65784_s_at', 'X76105_at'], 20: ['D00763_at', 'D14662_at', 'L36818_at', 'L42572_at', 'M19645_at', 'M23161_at', 'M34057_at', 'M69066_at', 'M75099_at', 'S79873_s_at', 'U01923_at', 'U06155_at', 'U07550_at', 'U66879_at', 'U68142_at', 'U9193

In [9]:
jpa = train_data[ train_data.iloc[:, -1] == "JPA" ]
jpa = jpa.T
jpa.drop(jpa.index[-1], inplace=True)
# jpa = jpa.T

jpa_inverse = train_data[ train_data.iloc[:, -1] != "JPA"]
jpa_inverse = jpa_inverse.T
jpa_inverse.drop(jpa_inverse.index[-1], inplace=True)
# jpa_inverse = jpa_inverse.T

print(jpa.shape, jpa_inverse.shape)

jpa["mean"] = jpa.mean(axis=1, skipna=True)
jpa["std"] = jpa.std(axis=1, skipna=True)
# print(jpa.head(3))

jpa_inverse["mean"] = jpa_inverse.mean(axis=1, skipna=True)
jpa_inverse["std"] = jpa_inverse.std(axis=1, skipna=True)
# print(jpa_inverse.head(3))

nume = jpa.loc[:,'mean'] - jpa_inverse.loc[:,'mean']
deno = np.sqrt( jpa.loc[:,'std'].pow(2)/6  + jpa_inverse.loc[:,'std'].pow(2)/64)

jpa.loc[:, 'T'] = nume / deno
jpa.drop(labels=['mean', 'std'], axis=1, inplace=True)
jpa.sort_values(['T'], ascending=False, na_position='last', inplace=True)
# jpa['JPA'] = "JPA"
jpa.drop(labels='T', axis=1, inplace=True)
# top_jpa = jpa.iloc[:30, :]
# top_jpa.sort_index(axis=0, inplace=True, na_position='last', ascending=True)
# # top_jpa.to_csv('jpa.csv')
# jpa_index = list(top_jpa.index)
top_jpas = {}
top_jpas_index = {}
N = [2, 4, 6, 8, 10, 12, 15, 20, 25, 30]
for item in N:
    top_jpas[item] = jpa.iloc[:item, :]
    top_jpas[item].sort_index(axis=0, inplace=True, na_position='last', ascending=True)
    top_jpas_index[item] = list(top_jpas[item].index)
print(top_jpas_index)

(6413, 6) (6413, 63)
{2: ['M16447_at', 'U79294_at'], 4: ['D13631_s_at', 'D25304_at', 'M16447_at', 'U79294_at'], 6: ['D13631_s_at', 'D25304_at', 'M16447_at', 'M93426_at', 'U51336_at', 'U79294_at'], 8: ['D13631_s_at', 'D25304_at', 'D87470_at', 'M16447_at', 'M93426_at', 'U51336_at', 'U79294_at', 'U90916_at'], 10: ['D13631_s_at', 'D25304_at', 'D87470_at', 'L08895_at', 'M16447_at', 'M93426_at', 'U51336_at', 'U79294_at', 'U90916_at', 'X61587_at'], 12: ['D13631_s_at', 'D25304_at', 'D87470_at', 'L08895_at', 'M16447_at', 'M93426_at', 'U51336_at', 'U79294_at', 'U90916_at', 'X03663_at', 'X60188_at', 'X61587_at'], 15: ['D13631_s_at', 'D25304_at', 'D87470_at', 'L08895_at', 'M16447_at', 'M93426_at', 'U24266_at', 'U32331_at', 'U48705_rna1_s_at', 'U51336_at', 'U79294_at', 'U90916_at', 'X03663_at', 'X60188_at', 'X61587_at'], 20: ['D13631_s_at', 'D25304_at', 'D87470_at', 'J04543_at', 'L08895_at', 'M16447_at', 'M93426_at', 'U24266_at', 'U32114_at', 'U32331_at', 'U48705_rna1_s_at', 'U51336_at', 'U52828_s_

In [10]:
# indexes = list(med_index)
# for item in mgl_index:
#     indexes.append(item)
# for item in epd_index:
#     indexes.append(item)
# for item in jpa_index:
#     indexes.append(item)
# for item in rhb_index:
#     indexes.append(item)
# print(len(indexes))

# indexes = list(set(indexes))
# print((indexes))

indexes = {}
N = [2, 4, 6, 8, 10, 12, 15, 20, 25, 30]
for item in N:
    indexes[item] = list(top_meds_index[item])
    for idx in top_mgls_index[item]:
        indexes[item].append(idx)
    for idx in top_epds_index[item]:
        indexes[item].append(idx)
    for idx in top_jpas_index[item]:
        indexes[item].append(idx)
    for idx in top_rhbs_index[item]:
        indexes[item].append(idx)
    indexes[item] = list(set(indexes[item]))

for item in indexes:
    print("{}:\n{}".format(item, indexes[item]))

2:
['D42041_at', 'M16447_at', 'X00371_rna1_at', 'D14662_at', 'M31303_rna1_at', 'U16954_at', 'U79294_at', 'X76105_at', 'J03191_at', 'V01515_cds1_at']
4:
['D00763_at', 'M57464_s_at', 'D85815_at', 'D80004_at', 'U16954_at', 'J03191_at', 'D42041_at', 'HG3514-HT3708_at', 'D13631_s_at', 'U79294_at', 'M31303_rna1_at', 'V01515_cds1_at', 'U79262_at', 'M16447_at', 'D14662_at', 'U07550_at', 'D25304_at', 'X00371_rna1_at', 'M20137_at', 'X76105_at']
6:
['D00763_at', 'M57464_s_at', 'D80004_at', 'D85815_at', 'D50310_at', 'U51336_at', 'X54304_at', 'U16954_at', 'U06155_at', 'J03191_at', 'L36818_at', 'D42041_at', 'HG3514-HT3708_at', 'X52011_at', 'D13631_s_at', 'U79294_at', 'M31303_rna1_at', 'V01515_cds1_at', 'U79262_at', 'M16447_at', 'D14662_at', 'D45248_at', 'U07550_at', 'M93119_at', 'D25304_at', 'U17760_rna1_at', 'X00371_rna1_at', 'M20137_at', 'M93426_at', 'X76105_at']
8:
['D00763_at', 'M57464_s_at', 'D80004_at', 'D85815_at', 'M75099_at', 'D50310_at', 'U51336_at', 'X54304_at', 'U16954_at', 'U06155_at', 

In [11]:
data = {}
N = [2, 4, 6, 8, 10, 12, 15, 20, 25, 30]
for n in N:
    data[n] = pd.DataFrame()
    for item in indexes[n]:
        data[n][[item]] = train_data[[item]]
    data[n]['class'] = train_data.iloc[:, -1]

# for item in indexes:
#     data[[item]] = train_data[[item]]
# data['class'] = train_data.iloc[:, -1]

data

C:\Users\Cypher\AppData\Local\Temp/ipykernel_31144/581191737.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[n]['class'] = train_data.iloc[:, -1]
C:\Users\Cypher\anaconda3\lib\site-packages\pandas\core\frame.py:3641: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]


{2:    D42041_at M16447_at X00371_rna1_at D14662_at M31303_rna1_at U16954_at  \
 1        178        89             29       243            724       931   
 2        103        88             20       228            579       845   
 3        182       106             20       185            608      1107   
 4        249        78             28       238            619      1187   
 5        200        75             33       173            676       969   
 ..       ...       ...            ...       ...            ...       ...   
 65       221       253             35       309            343       352   
 66       209       228             43       249            277       329   
 67       173       220             31       294            256       313   
 68       163       233             33       285            271       287   
 69       187       200             39       356            228       269   
 
    U79294_at X76105_at J03191_at V01515_cds1_at class  
 1         99 

In [ ]:
# data.to_csv('pp5i_train.topN.gr.csv')
N = [2, 4, 6, 8, 10, 12, 15, 20, 25, 30]
for item in N:
    data[item].to_csv('pp5i_train.top{}.gr.csv'.format(item))

In [ ]:
import pandas as pd
import numpy as np


train = pd.read_csv('pp5i_train.topN.gr.csv')

train.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
print(train.shape, "\n", train )
